# Validate configuration for reproducing "Revisiting TCP Congestion Control Throughput Models & Fairness Properties at Scale"

## Set up your FABRIC environment


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,vinita_p_0000073925
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [2]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [3]:
n_endpoints = 10
slice_name="bottleneck-" + str(n_endpoints) + '-test'

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [4]:
slice = fablib.get_slice(name=slice_name)

In [5]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [6]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

## Check network capacity

We should get close to 25 Gbps (probably more like 23 Gbps because of protocol overhead) through the router.

First, make sure there is no rate limiting at the router - don't worry if you get an error message, this just means there was already no rate limiting.

In [7]:
router_node.execute("sudo tc qdisc del dev " + router_egress_name + " root")

 Error: Cannot delete qdisc with handle of zero.
 

('', 'Error: Cannot delete qdisc with handle of zero.\n')

Change the queue at the sender nodes to pfifo_fast to replace existing fq_codel queue

In [10]:
for n in sender_nodes:
    sender_inf=n.get_interface(network_name= "link-sender")
    sender_inf_name = sender_inf.get_device_name()
    n.execute("sudo tc qdisc replace dev " + sender_inf_name + " root handle 1: pfifo_fast")
    n.execute("tc -s -d qdisc show dev "+ sender_inf_name)


qdisc pfifo_fast 1: root refcnt 17 bands 3 priomap  1 2 2 2 1 2 0 0 1 1 1 1 1 1 1 1
 Sent 0 bytes 0 pkt (dropped 0, overlimits 0 requeues 0) 
 backlog 0b 0p requeues 0
qdisc pfifo_fast 1: root refcnt 17 bands 3 priomap  1 2 2 2 1 2 0 0 1 1 1 1 1 1 1 1
 Sent 0 bytes 0 pkt (dropped 0, overlimits 0 requeues 0) 
 backlog 0b 0p requeues 0
qdisc pfifo_fast 1: root refcnt 17 bands 3 priomap  1 2 2 2 1 2 0 0 1 1 1 1 1 1 1 1
 Sent 0 bytes 0 pkt (dropped 0, overlimits 0 requeues 0) 
 backlog 0b 0p requeues 0
qdisc pfifo_fast 1: root refcnt 17 bands 3 priomap  1 2 2 2 1 2 0 0 1 1 1 1 1 1 1 1
 Sent 0 bytes 0 pkt (dropped 0, overlimits 0 requeues 0) 
 backlog 0b 0p requeues 0
qdisc pfifo_fast 1: root refcnt 17 bands 3 priomap  1 2 2 2 1 2 0 0 1 1 1 1 1 1 1 1
 Sent 0 bytes 0 pkt (dropped 0, overlimits 0 requeues 0) 
 backlog 0b 0p requeues 0
qdisc pfifo_fast 1: root refcnt 17 bands 3 priomap  1 2 2 2 1 2 0 0 1 1 1 1 1 1 1 1
 Sent 0 bytes 0 pkt (dropped 0, overlimits 0 requeues 0) 
 backlog 0b 0p req

Now start the `iperf3` flows:

In [14]:
for n in receiver_nodes:
    n.execute("sudo killall iperf3")
    n.execute("iperf3 -s -1 -f g -D --logfile validate.dat")

In [15]:
import time
for i,n in enumerate(sender_nodes):
    n.execute("sudo killall iperf3")
    n.execute_thread("iperf3 -t 60 -P 10 -c 10.10.2.1"+str(i))
time.sleep(65)

 iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
  iperf3: no process found
 

In [16]:
tputs = []
for n in receiver_nodes:
    (nout, nerr) = n.execute("tail --lines=2 validate.dat | grep receiver | awk '{print $6}'")
    print(nout)
    tputs.append(float(nout.strip()))
print("Sum throughput is: %f Gbps" % sum(tputs))

3.07
3.07

2.99
2.99

3.04
3.04

2.83
2.83

2.56
2.56

2.57
2.57

2.26
2.26

2.48
2.48

3.55
3.55

2.90
2.90

Sum throughput is: 28.250000 Gbps


A single pair of endpoints should be able to get about 10 Gbps, possibly higher.

In [18]:
receiver_nodes[0].execute("iperf3 -s -1 -f g -D")
sender_nodes[0].execute("iperf3 -t 60 -i 60 -P 10 -c 10.10.2.10")

Connecting to host 10.10.2.10, port 5201
[  5] local 10.10.1.10 port 58394 connected to 10.10.2.10 port 5201
[  7] local 10.10.1.10 port 58402 connected to 10.10.2.10 port 5201
[  9] local 10.10.1.10 port 58410 connected to 10.10.2.10 port 5201
[ 11] local 10.10.1.10 port 58412 connected to 10.10.2.10 port 5201
[ 13] local 10.10.1.10 port 58418 connected to 10.10.2.10 port 5201
[ 15] local 10.10.1.10 port 58430 connected to 10.10.2.10 port 5201
[ 17] local 10.10.1.10 port 58438 connected to 10.10.2.10 port 5201
[ 19] local 10.10.1.10 port 58442 connected to 10.10.2.10 port 5201
[ 21] local 10.10.1.10 port 58444 connected to 10.10.2.10 port 5201
[ 23] local 10.10.1.10 port 58448 connected to 10.10.2.10 port 5201
[ ID] Interval           Transfer     Bitrate         Retr  Cwnd
[  5]   0.00-60.00  sec  12.1 GBytes  1.73 Gbits/sec  296    199 KBytes       
[  7]   0.00-60.00  sec  12.1 GBytes  1.73 Gbits/sec  358    209 KBytes       
[  9]   0.00-60.00  sec  12.1 GBytes  1.73 Gbits/sec  29

('Connecting to host 10.10.2.10, port 5201\n[  5] local 10.10.1.10 port 58394 connected to 10.10.2.10 port 5201\n[  7] local 10.10.1.10 port 58402 connected to 10.10.2.10 port 5201\n[  9] local 10.10.1.10 port 58410 connected to 10.10.2.10 port 5201\n[ 11] local 10.10.1.10 port 58412 connected to 10.10.2.10 port 5201\n[ 13] local 10.10.1.10 port 58418 connected to 10.10.2.10 port 5201\n[ 15] local 10.10.1.10 port 58430 connected to 10.10.2.10 port 5201\n[ 17] local 10.10.1.10 port 58438 connected to 10.10.2.10 port 5201\n[ 19] local 10.10.1.10 port 58442 connected to 10.10.2.10 port 5201\n[ 21] local 10.10.1.10 port 58444 connected to 10.10.2.10 port 5201\n[ 23] local 10.10.1.10 port 58448 connected to 10.10.2.10 port 5201\n[ ID] Interval           Transfer     Bitrate         Retr  Cwnd\n[  5]   0.00-60.00  sec  12.1 GBytes  1.73 Gbits/sec  296    199 KBytes       \n[  7]   0.00-60.00  sec  12.1 GBytes  1.73 Gbits/sec  358    209 KBytes       \n[  9]   0.00-60.00  sec  12.1 GBytes  1.

## Check network delay

We should get sub-ms latency across the router.

In [ ]:
for i,n in enumerate(sender_nodes):
    n.execute("ping -c 5 10.10.2.1"+str(i)+" | grep rtt")

rtt min/avg/max/mdev = 0.110/0.153/0.266/0.059 ms
rtt min/avg/max/mdev = 0.116/0.157/0.297/0.071 ms
